In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import VWAP, SimpleMovingAverage
from quantopian.pipeline import CustomFilter
import numpy as np
import matplotlib.pyplot as plt
start = '2015-6-1'
end = '2016-7-1'
TICK = 'SPY'

In [ ]:
class SidInList(CustomFilter):  
    """  
    Filter returns True for any SID included in parameter tuple passed at creation.  
    Usage: my_filter = SidInList(sid_list=(23911, 46631))  
    """  
    inputs = []  
    window_length = 1  
    params = ('sid_list',)

    def compute(self, today, assets, out, sid_list):  
        out[:] = np.in1d(assets, sid_list)

In [ ]:
def make_pipeline():
    
    vwap = VWAP(inputs=[USEquityPricing.close, USEquityPricing.volume], window_length=14)
    
    short_sma = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=35)
    long_sma = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=50)
    
    
    close = USEquityPricing.close.latest
    symbol = SidInList(sid_list=(8554))
    
    return Pipeline(
        columns={
            'vwap': vwap,
            'short_sma': short_sma,
            'long_sma': long_sma,
            'vol': USEquityPricing.volume.latest,
            'close': close,
            'symbol': symbol,
       },
       screen=symbol,
    )

In [ ]:
result = run_pipeline(make_pipeline(), start, end)
result.head()

In [ ]:
dates = [x[0].to_datetime() for x in result.index.tolist()]

In [ ]:
price = result.close.values


In [ ]:
vwap_values = result.vwap.values
#upper_vwap = [val*1.01 for val in vwap_values]
#lower_vwap = [val*0.99 for val in vwap_values]

short_sma_vals = result.short_sma.values
long_sma_vals = result.long_sma.values

In [ ]:
#plt.plot(dates, vwap_values, color='r')
plt.plot(dates, short_sma_vals, color='r')
plt.plot(dates, long_sma_vals, color='b')
plt.plot(dates, price, color='c')

#plt.plot(dates, upper_vwap)
#plt.plot(dates, lower_vwap)

In [ ]:
vol = result.vol.values
plt.plot(dates, [vol.mean()+0.5*vol.std()]*len(dates))
plt.plot(dates, [vol.mean()]*len(dates))
plt.plot(dates, [vol.mean()-0.5*vol.std()]*len(dates))
plt.plot(dates, vol)

In [ ]:
print vol.std()
print vol.mean()

In [ ]:
# WITH PIPELINE

In [ ]:
# WITHOUT PIPELINE INTRADAY

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage 
from quantopian.pipeline import CustomFilter

In [ ]:
date = '2016-9-28'
hist2 = get_pricing(TICK, start_date=date, end_date=date, frequency='minute')

p = hist2['price']
v = hist2['volume']
h = hist2['high']
l = hist2['low']

hist2['sma'] = pd.rolling_mean(p, window=15)
hist2['eqma'] = pd.ewma(p, span=20)
hist2['vwap'] = (v*(h+l)/2).cumsum() / v.cumsum()
hist2

In [ ]:
plt.plot(p.index, p.values, color='c')
#plt.plot(hist2['sma'].index, hist2['sma'].values, color='b')
#plt.plot(hist2['eqma'].index, hist2['eqma'].values, color='B')
plt.plot(hist2['vwap'].index, hist2['vwap'].values, color='r')
plt.ylabel('Price')
#plt.legend([TICK, 'EQMA', 'VWAP'])

In [ ]:
p.values

In [ ]:
p.index

In [ ]:
len(p.values) == len(p.index)

In [ ]:
e = pd.ewma(p, span=20)

In [ ]:
e[-2:-1]

In [ ]:
e[-2]

In [ ]:
e[-1]